<a href="https://colab.research.google.com/github/mkane968/Text-Mining-Experiments/blob/main/Text_Sectioning_and_Disaggregation_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Intro to Text Sectioning and Disaggregation

This code can be used to clean, section, and disaggregate texts and corpora. Dividing texts into sections (here, chapters or chunks of N lenth) is particularly valuable as a precursor to topic modeling and other forms of computational analysis which perform more accurately when applied to groups of segmented documents from longer texts. 

The process of disaggregating the words in texts (in this case, by alphabetizing them) also creates data sets that can be shared freely where original texts may not be due to copyright restrictions. 

This code requires plain txt files as input, either those from this repository's sample_data folder or those from a local machine. It returns csv files with disaggregated text grouped by chapter or chunk of n length.

#Upload and Add Text Files To Pandas DataFrame
In this section, text files are mounted to Google Drive from the local machine and then loaded into a Pandas DataFrame. Pandas is a fast and relatively easy way to work with large datasets. Though data frames are typically associated with numbers, Pandas also offers many functionalities for [working with textual data. ](https://www.tutorialspoint.com/python_pandas/python_pandas_working_with_text_data.htm) 

In [ ]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Selet all files to upload
from google.colab import files

uploaded = files.upload()

In [ ]:
#Add files into dataframe
import pandas as pd

books = pd.DataFrame.from_dict(uploaded, orient='index')
books

In [ ]:
#Reset index and add column names to make wrangling easier
books = books.reset_index()
books.columns = ["Title", "Text"]
books

In [ ]:
#Remove encoding characters from Text column (b'\xef\xbb\xbf)
books['Text'] = books['Text'].apply(lambda x: x.decode('utf-8'))

In [ ]:
#Change data type to string (to support further cleaning below)
books = books.astype(str)

#Clean Texts and Set Parameters for Sectioning
Several basic cleaning processes are implemented: removing unwanted characters from titles, removing newline characters from texts, and removing punctuation. Parameters are also set for part(s) of text to be included in sectioning. In the SciFi Corpus project, "START OF BOOK" and "END OF BOOK" tags were added to delineate the body of each text. Code in this section removes any text outside the starting and ending parameters--e.g., title page, copyright page, other paratext. 

In [ ]:
#Remove .txt from titles
books['Title'] = books['Title'].str.replace(r'.txt', ' ', regex=True) 
books

In [ ]:
#Remove newline characters
books['Text'] = books['Text'].str.replace(r'\s+|\\r', ' ', regex=True) 
books['Text'] = books['Text'].str.replace(r'\s+|\\n', ' ', regex=True) 
books

In [ ]:
#Remove punctuation
books['Text'] = books['Text'].str.replace(r'[^\w\s]+', '', regex = True)

In [ ]:
#Remove paratext (before and after START OF BOOK and END OF BOOK tags)
#If texts you are working with do not have these tags, ignore this cell

#Split book on start of book tag, keep text only after start of book tag
start = books["Text"].str.split("START OF BOOK", expand = True)
books['Text'] = start[1]

#Split book on end of book tag, keep text only before of book tag
end = books["Text"].str.split("END OF BOOK", expand = True)
books['Text'] = end[0]
books

In [ ]:
#Check that text is cleaned and sectioned
books.iloc[0]['Text']

In [ ]:
#Define new dataframe
books_cleaned = books

#Section Texts By Chapter Headings
When working with texts with clearly delineated chapters, using chapter headings is a relatively easy way to section texts into segments of (relatively) the same size. After checking the chapter counts for each text to confirm whether sectioning by chapter is a useful procedure, this code iterates through the texts and splits them each time it encounters a new "chapter" heading. From here, the text from each chapter is appended to a new dataframe and denoted by book and chapter number. 

In [ ]:
#Count number of chapters in each text
chapter_counts = books_cleaned['Text'].str.count('CHAPTER')

#Append chapter counts to dataframe
books_cleaned["Chapters"] = chapter_counts
books_cleaned

In [ ]:
#Make new cell each time new chapter starts 
new = books_cleaned["Text"].str.split("CHAPTER", expand = True).set_index(books_cleaned['Title'])
new

In [ ]:
#Flatten dataframe so each chapter is on own row, designated by book and chapter 
chapters_df = new.stack().reset_index()
chapters_df.columns = ["Book", "Chapter", "Text"]
chapters_df.dropna
chapters_df

In [ ]:
#Tidying the DF
#Combine book and chapter labels into one column
chapters_df['Book + Chapter'] = chapters_df['Book'].astype(str) + '_Chapter_' + chapters_df['Chapter'].astype(str)

#Remove individual book and chapter columns
chapters_df.drop(columns=['Book', 'Chapter'])

#Reindex so book + chapter is first column 
column_names = "Book + Chapter", "Text"
chapters_df = chapters_df.reindex(columns=column_names)
chapters_df

#Section Texts By Chunk of N Length
When working with texts WITHOUT discernable chapter headings--or, even if chapter headings are present but too infrequent to split texts into meaningful segments--texts can instead be sectioned by chunks of "N" length, where N is a variable that can be custom-set below. After checking the word counts for each text to determine what size chunks would be appropriate, this code iterates through the texts and splits them each time it counts "N" number of words. From here, the text from each chunk is appended to a new dataframe and denoted by book and chunk number.

In [ ]:
#Get number of words in each book (helps to determine chunk length)
words = books_cleaned["Text"].apply(lambda x: len(str(x).split(' ')))

#Append chapter counts to dataframe
books_cleaned["Word Count"] = words
books_cleaned

In [ ]:
#Tokenize Text
import nltk
nltk.download('punkt')
books_cleaned['Tokens'] = books_cleaned.apply(lambda row: nltk.word_tokenize(row['Text']), axis=1)
books_cleaned

In [ ]:
#Define chunking function
def split(list_a, chunk_size):
  for i in range(0, len(list_a), chunk_size):
    yield list_a[i:i + chunk_size]

#Set desired size of chunks
chunk_size = 1000

#Create new list for chunked sentences
chunked_sentences = []

#Perform chunking function on each row of tokens
s = books_cleaned['Tokens']
for content in s:
  chunks = list(split(content, chunk_size))
  #Check that text is being chunked correctly
  print(chunks[0])
  #Convert from tokens to sentences to add to new df
  for chunk in chunks:
    sentences = ' '.join(chunk)
    chunked_sentences.append(sentences)

In [ ]:
#Create new dataframe to store chunked sentences for disaggregation
#NEEDS WORK--"Title" does not reflect which chunks belong to which books
df = pd.DataFrame(chunked_sentences).T
chunked_df = df.stack().reset_index()
chunked_df.columns = ["Title","Chunk","Text"]
chunked_df

In [ ]:
#Tidying the DF
#Combine book and chunk labels into one column
chunked_df['Book + Chunk'] = chunked_df['Title'].astype(str) + ' Chunk ' + chunked_df['Chunk'].astype(str)

#Remove individual book and chunk columns
chunked_df.drop(columns=['Title', 'Chunk'])

#Reindex so book + chunk is first column 
column_names = "Book + Chunk", "Text"
chunked_df = chunked_df.reindex(columns=column_names)
chunked_df

#Disaggregate Texts and Download CSV Output
Working with texts split by chapter or chunk, the final step of this process is to disaggregate the data. Disaggregation, or the breakdown of data into smaller (disordered) parts, is accomplished through the alphabetization of the words in each chapter/chunk. 

The resulting "bag of words" data can then be downloaded as csvs and used for further analysis, such as through the Topic Modeling pipeline in the Extracted Features repository: https://github.com/SF-Nexus/Extracted-Features/blob/main/Topic%20Modeling%20with%20SciFi%20Corpus.ipynb 

In [ ]:
#Working with data from texts sectioned by CHAPTER
#Alphabetize words in each chapter string
chapters_df['Text'] = chapters_df['Text'].apply(lambda x: ' '.join(sorted(x.split())))
chapters_df

In [ ]:
#Download disaggregated chapters to csv
from google.colab import files

chapters_df.to_csv('chapters_bag_of_words_output.csv', encoding = 'utf-8-sig') 
files.download('chapters_bag_of_words_output.csv')

In [ ]:
#Working with data from texts sectioned by CHUNK of N length
#Alphabetize words in each chunk string
chunked_df['Text'] = chunked_df['Text'].apply(lambda x: ' '.join(sorted(x.split())))
chunked_df

In [ ]:
#Download disaggregated chunks to csv
from google.colab import files

chunked_df.to_csv('word_chunks_bag_of_words_output.csv', encoding = 'utf-8-sig') 
files.download('word_chunks_bag_of_words_output.csv')